## I

# Imports

In [1]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments,Trainer
from datasets import load_dataset,DatasetDict
import torch


device = "cuda" if torch.cuda.is_available() else "cpu"

# Loading model, tokenizer and splitting the dataset

The task at hand that I have first is to load the model, its tokenizer and then the dataset, but also I have to split the dataset, changes it labels, becuase labels are in string format rather than in number, so have to change it using ClassLabel and then also to change the name of the columns to follow the format that the `Trainer` class expects

In [ ]:
from datasets import ClassLabel

# constants I will use later on
data_path = "/Users/kannavsethi/Desktop/nlp-final-project/data/AI_Human.csv"
text_column = "text"
label_column = "generated"
model_name = "distilbert-base-uncased"

raw_data = load_dataset("csv", data_files={"full": data_path})

# renaming the columns
raw_data = raw_data.rename_column(label_column, "label")

# what this does is to convert the label column into a ClassLabel type
# which is a special type used by the datasets library to handle labels
labels = raw_data["full"].unique("label")
raw_data = raw_data.cast_column("label", ClassLabel(names=labels))

# loading the full dataset and then using the train_test_split method to split it into two halfs, testing and training
d0 = raw_data["full"].train_test_split(test_size=0.2, seed=42)

# same thing, but now testing and validation
d1 = d0["test"].train_test_split(test_size=0.5, seed=42)

# creating a DatasetDict to hold the train, validation, and test sets, much easier for me to look into it
dataset = DatasetDict({
    "train": d0["train"],       # 80%
    "validation": d1["train"],  # 10%
    "test": d1["test"],         # 10%
})

Generating full split: 0 examples [00:00, ? examples/s]

Casting the dataset:   0%|          | 0/487235 [00:00<?, ? examples/s]

# Creating the custom tokenizer function that we will be using for batched inputs

In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, return_tensors="pt")

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/389788 [00:00<?, ? examples/s]

Map:   0%|          | 0/48723 [00:00<?, ? examples/s]

Map:   0%|          | 0/48724 [00:00<?, ? examples/s]

# Trainer and Training Arguments

To fine-tune, I will be using the Trainer class from transformers and corresponding training_arguments class as well to specify parameters for the same

In [ ]:
# metrics that i made in a separate file as they are used in multiple places, so wanted them to be modular
from evaluation_metrics import compute_metrics_for_trainer, evaluate_model

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)
training_args = TrainingArguments(
    output_dir = "./models", # place to store the model checkpoints
    eval_strategy="epoch", # evaluate the model at the end of each epoch
    learning_rate=2e-5, # learning rate for the optimizer
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16, # batch size for evaluation
    num_train_epochs=3, # number of training epochs
    weight_decay=0.01, # weight decay for regularization
    load_best_model_at_end=True, # load the best model at the end of training
    logging_dir="./logs", # directory for storing logs
    logging_steps=10, # log every 10 steps
    save_strategy="epoch", # save the model at the end of each epoch
    report_to="none" # i don't want the wandb thing to show up, so I set it to none
)


trainer = Trainer(
    model=model, # the model to train
    args=training_args, # the training arguments
    train_dataset=tokenized_datasets["train"], # the training dataset
    eval_dataset=tokenized_datasets["validation"], # the validation dataset
    tokenizer=tokenizer, # the tokenizer to use
    compute_metrics=compute_metrics_for_trainer # the function to compute metrics during evaluation, again this is the custom function that I made
)

trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_36/2783823326.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.000100,0.002568,0.999405,0.999201,0.999339,0.999063
2,0.000000,0.001430,0.999651,0.999532,0.999669,0.999394
3,0.000000,0.000722,0.999815,0.999752,0.999780,0.999725


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=36543, training_loss=0.00578049455844403, metrics={'train_runtime': 35565.9295, 'train_samples_per_second': 32.879, 'train_steps_per_second': 1.027, 'total_flos': 1.549026071619748e+17, 'train_loss': 0.00578049455844403, 'epoch': 3.0})

# Finally evaluating and seeing detailed metrics 

In [ ]:

# evaluating the model on the test set
results = trainer.evaluate(tokenized_datasets["test"])
print("Test Results:", results)

# making predictions on the test set
predictions = trainer.predict(tokenized_datasets["test"])
y_pred = predictions.predictions.argmax(-1)
y_true = predictions.label_ids

# printing the classification report
print("\nDetailed Test Metrics:")
evaluate_model(y_true, y_pred)

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Test Results: {'eval_loss': 0.0013374830596148968, 'eval_accuracy': 0.9997331910352187, 'eval_f1': 0.9996445076430859, 'eval_precision': 0.9996171725458026, 'eval_recall': 0.9996718442353971, 'eval_runtime': 469.88, 'eval_samples_per_second': 103.695, 'eval_steps_per_second': 3.241, 'epoch': 3.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Detailed Test Metrics:
Accuracy: 0.9997331910352187
F1 Score: 0.9996445076430859
Precision: 0.9996171725458026
Recall: 0.9996718442353971


{'accuracy': 0.9997331910352187,
 'f1_score': 0.9996445076430859,
 'precision': 0.9996171725458026,
 'recall': 0.9996718442353971}